Transformer Architecture.
In general terms Transformer consists of Encoder and Decoder parts.


## The Illustrated Transformer

Attention:

## Self-Attention.
Calculation of Self-Attension consists of next parts:
* 1) The first step in calculating self-attention is to create three vectors from each of the encoder’s input vectors (in this case, the embedding of each word). So for each word, we create a **Query vector, a Key vector, and a Value vector**. These vectors are created by multiplying the embedding by three matrices that we trained during the training process.

Notice that these new vectors are smaller in dimension than the embedding vector. Their dimensionality is 64, while the embedding and encoder input/output vectors have dimensionality of 512. They don’t HAVE to be smaller, this is an architecture choice to make the computation of multiheaded attention (mostly) constant.

Example:
Input -> Embeddings (words to $X^1, ... , X^n$

Embeddings -> $X1 \cdot W^Q $  (product $Embeddings \cdot W^Q
matrix) -> query$

Embeddings -> $X1 \cdot W^K $  (product $Embeddings \cdot W^K
matrix) -> key$

Embeddings -> $X1 \cdot W^V $  (product $Embeddings \cdot W^V
matrix) -> value$

* 2) The second step in calculating self-attention is to calculate a score. Say we’re calculating the self-attention for the first word in this example, “Thinking”. We need to score each word of the input sentence against this word. The score determines how much focus to place on other parts of the input sentence as we encode a word at a certain position.

The score is calculated by taking the **dot product of the query vector with the key vector of the respective word we’re scoring**. So if we’re processing the self-attention for the word in position #1, the first score would be the dot product of q1 and k1. The second score would be the dot product of q1 and k2.

* 3) The third and forth steps are to divide the scores by 8 (the square root of the dimension of the key vectors used in the paper – 64. This leads to having more stable gradients. There could be other possible values here, but this is the default), then pass the result through a softmax operation. **Softmax normalizes the scores so they’re all positive and add up to 1.**

* 4) The fifth step is to multiply each value vector by the softmax score (in preparation to sum them up). The intuition here is to keep intact the values of the word(s) we want to focus on, and drown-out irrelevant words (by multiplying them by tiny numbers like 0.001, for example).

* 5) The sixth step is to sum up the weighted value vectors. This produces the output of the self-attention layer at this position (for the first word).

## Matrix Calculation of Self-Attention:
* The first step is to calculate the **Query, Key, and Value** matrices. We do that by packing our embeddings into a matrix X, and multiplying it by the weight matrices we’ve trained ($W^Q, W^K, W^V$).

$
\begin{equation*}
X =
\begin{pmatrix}
X^1 \\
X^2 \\
... \\
X^d
\end{pmatrix}
\end{equation*}
$ where $X^1, X^2, ..., X^d$ - $Embeddings$

$X \bullet W^Q = Q$, 
$
\begin{equation*}
Q =
\begin{pmatrix}
Q^1 \\
Q^2 \\
... \\
Q^d
\end{pmatrix}
\end{equation*}
$

$X \bullet W^K = K$,
$
\begin{equation*}
K =
\begin{pmatrix}
K^1 \\
K^2 \\
... \\
K^d
\end{pmatrix}
\end{equation*}
$

$X \bullet W^V = V$,
$
\begin{equation*}
V =
\begin{pmatrix}
V^1 \\
V^2 \\
... \\
V^d
\end{pmatrix}
\end{equation*}
$

Finally:

$softmax(\frac{Q \bullet K^T}{\sqrt{d_k}}) \cdot V = Z$,  
where  $softmax = \sigma(z)_i = \frac{e^{z_i}}{\sum^K_{k=1}{e^{z_k}}}$

## Multi-Headed Attention
The paper further refined the self-attention layer by adding a mechanism called “multi-headed” attention. This improves the performance of the attention layer in two ways:

    * 1) It expands the model’s ability to focus on different positions. Yes, in the example above, z1 contains a little bit of every other encoding, but it could be dominated by the the actual word itself. It would be useful if we’re translating a sentence like “The animal didn’t cross the street because it was too tired”, we would want to know which word “it” refers to.

    * 2) It gives the attention layer multiple “representation subspaces”. As we’ll see next, with multi-headed attention we have not only one, but multiple sets of Query/Key/Value weight matrices (the Transformer uses eight attention heads, so we end up with eight sets for each encoder/decoder). Each of these sets is randomly initialized. Then, after training, each set is used to project the input embeddings (or vectors from lower encoders/decoders) into a different representation subspace.

In multi-attention paradigm:

1) Input sequence.

2) Embed each word to $X$

* In all encoders other than 0, we don't need embedding. We start directly with the output of the encoder right velow this one.

3) We have multiple $(Q_0, K_0, V_0), ..., (Q_0, K_n, V_n)$. We multiply $X$ or $R$ with weigh matrices.

4) Calculate attention using the resulting $Q$/$K$/$V$ matrices. Then we have $(Z_1, Z_2, ..., Z_n)$ attentions.

5) Now we concatenate all $Z_i$ and multiply concatenation on $W^0$ matrix.

* Where $W^0$ is a weights matrix, that was trained with model.

## Trasformer Encoder:
$(X_1, X_2, ..., X_n) \to Self-Attention \to Add-Normalize(X + Z) =  P \to Feed-Forward \to Add-Normalize(P + Feed-Forward-output)$

## Transformer Decoder:
The self attention layers in the decoder operate in a slightly different way than the one in the encoder:

In the decoder, the self-attention layer is only allowed to attend to earlier positions in the output sequence. This is done by masking future positions (setting them to -inf) before the softmax step in the self-attention calculation.

The “Encoder-Decoder Attention” layer works just like multiheaded self-attention, except it creates its Queries matrix from the layer below it, and takes the Keys and Values matrix from the output of the encoder stack.

## The Final Linear and Softmax Layer

The decoder stack outputs a vector of floats. How do we turn that into a word? That’s the job of the final Linear layer which is followed by a Softmax Layer.

The Linear layer is a simple fully connected neural network that projects the vector produced by the stack of decoders, into a much, much larger vector called a logits vector.

Let’s assume that our model knows 10,000 unique English words (our model’s “output vocabulary”) that it’s learned from its training dataset. This would make the logits vector 10,000 cells wide – each cell corresponding to the score of a unique word. That is how we interpret the output of the model followed by the Linear layer.

The softmax layer then turns those scores into probabilities (all positive, all add up to 1.0). The cell with the highest probability is chosen, and the word associated with it is produced as the output for this time step.

Decoder stack output $\to Liner \to logits \to Softmax \to logprobs \to argmax \to word$

$ \vec{logits} = (0, 1, 2, ..., vocab\_size)$

## Loss function

Say we are training our model. Say it’s our first step in the training phase, and we’re training it on a simple example – translating “merci” into “thanks”.

What this means, is that we want the output to be a probability distribution indicating the word “thanks”. But since this model is not yet trained, that’s unlikely to happen just yet.

How do you compare two probability distributions? We simply subtract one from the other. For more details, look at cross-entropy and Kullback–Leibler divergence.

But note that this is an oversimplified example. More realistically, we’ll use a sentence longer than one word. For example – input: “je suis étudiant” and expected output: “i am a student”. What this really means, is that we want our model to successively output probability distributions where:

Each probability distribution is represented by a vector of width vocab_size (6 in our toy example, but more realistically a number like 3,000 or 10,000)
The first probability distribution has the highest probability at the cell associated with the word “i”
The second probability distribution has the highest probability at the cell associated with the word “am”
And so on, until the fifth output distribution indicates ‘<end of sentence>’ symbol, which also has a cell associated with it from the 10,000 element vocabulary.
    
Now, because the model produces the outputs one at a time, we can assume that the model is selecting the word with the highest probability from that probability distribution and throwing away the rest. That’s one way to do it (called greedy decoding). Another way to do it would be to hold on to, say, the top two words (say, ‘I’ and ‘a’ for example), then in the next step, run the model twice: once assuming the first output position was the word ‘I’, and another time assuming the first output position was the word ‘a’, and whichever version produced less error considering both positions #1 and #2 is kept. We repeat this for positions #2 and #3…etc. This method is called “beam search”, where in our example, beam_size was two (because we compared the results after calculating the beams for positions #1 and #2), and top_beams is also two (since we kept two words). These are both hyperparameters that you can experiment with.

**Links**

[Transformers from Scratch](http://www.peterbloem.nl/blog/transformers)  


## BERT